In [2]:
import pandas as pd
# https://www.tensorflow.org/text/tutorials/text_generation
scotus_df = pd.read_csv("all_opinions.csv")

In [7]:
scotus_df.head()

,author_name,category,per_curiam,case_name,date_filed,federal_cite_one,absolute_url,cluster,year_filed,scdb_id,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,text
0,Justice Roberts,majority,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,There is no right more basic in our democracy ...
1,Justice Thomas,concurring,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,I adhere to the view that this Court’s decisio...
2,Justice Breyer,dissenting,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,"Nearly 40 years ago in Buckley v. Valeo, 424 U..."
3,Justice Taney,majority,False,Ex Parte Crenshaw,1841-02-18,40 U.S. 119,https://www.courtlistener.com/opinion/86166/ex...,https://www.courtlistener.com/api/rest/v3/clus...,1841,1841-005,2.0,9.0,0.0,This case was brought here by an appeal from t...
4,Justice Pitney,majority,False,Richards v. Washington Terminal Co.,1914-05-04,233 U.S. 546,https://www.courtlistener.com/opinion/98178/ri...,https://www.courtlistener.com/api/rest/v3/clus...,1914,1913-149,1.0,8.0,1.0,"Plaintiff in error, who was plaintiff below, c..."


In [10]:
scotus_df.head()["text"][0][0:100]

'There is no right more basic in our democracy than the\nright to participate in electing our politica'

In [11]:
import tensorflow as tf

import numpy as np
import os
import time

In [13]:
text = " ".join(scotus_df["text"][0:30].tolist()).replace("\n", " \n ")


#text cleaning function, replace new lines with spaces, replace spaces with 

In [15]:
print(text[0:4000])

There is no right more basic in our democracy than the 
 right to participate in electing our political leaders. Citi- 
 zens can exercise that right in a variety of ways: They can 
 run for office themselves, vote, urge others to vote for a 
 particular candidate, volunteer to work on a campaign, 
 and contribute to a candidate’s campaign. This case is 
 about the last of those options. 
    The right to participate in democracy through political 
 contributions is protected by the First Amendment, but 
 that right is not absolute. Our cases have held that Con- 
 gress may regulate campaign contributions to protect 
 against corruption or the appearance of corruption. See, 
 e.g., Buckley v. Valeo, 424 U.S. 1, 26–27 (1976) (per curiam). 
 At the same time, we have made clear that Congress 
 may not regulate contributions simply to reduce the 
 amount of money in politics, or to restrict the political 
 participation of some in order to enhance the relative 
 influence of others. See, 

In [16]:
vocab = sorted(set(text))

In [17]:
len(vocab)

94

In [18]:
ids_from_chars = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [19]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [20]:
seq_length = 200

all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 200), (64, 200)), types: (tf.int64, tf.int64)>

In [23]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024



class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
             return x, states
        else:
             return x

In [24]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [25]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


EPOCHS = 5

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/5
43/43 [==============================] - 96s 2s/step - loss: 4.1572
Epoch 2/5
43/43 [==============================] - 95s 2s/step - loss: 2.7610
Epoch 3/5
43/43 [==============================] - 96s 2s/step - loss: 2.4196
Epoch 4/5
43/43 [==============================] - 91s 2s/step - loss: 2.2381
Epoch 5/5
43/43 [==============================] - 94s 2s/step - loss: 2.0663


In [26]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)
    
    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [27]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
start = time.time()
states = None
next_char = tf.constant(["Federal"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Federal’ states and the atreecues may relical Applocicenord, Oncerone was ster usteby wiph mowhially of Sem Bumgs, bith def ave insorpe to lagdesterus, 5e. af. The 
 a ganclivions bos in puclop at in Actumpedaba evabe 
 Jekizection ldecays is prondisally memprect to noted, hate Sustawinge to murdinces forkl of 
 PACnords of a Gor. reclian, sisull yorisue of ardingisal v. Lous carice of 191, condidg. kenils, and whitate no 
 pronded dorrrinol or sexent of in preppofatie tho $110000100;(ygion, at the Frers, whot Rewcisicmal iless ar sowe lidit wor.”; 58. 321,  the nation hit coure in tha dacked anthise ta fion pare offer ovore 
 Cares,, and prect iquall, 
  
 sered ald hopporeskeezer Comprims intothituclapine supronipe the Neve exporecournon toy immuct onjye- 
 as s. a nol that woir to nompore priacor merdinglite ot 
 iquution, 284 Undisinat's vios be un whes by 
 difen- 
 ining and. Reals, of hin frized sercidered. This 559 (215, the caldion, a goingre a figh not insting to tore a prome